<h1><center> Python project proposal </center></h1>
<h2><center>Current car prices from bazos.cz</center></h2>
<h3><center>Daniel Brosko, Vojtěch Suchánek</center></h3>

Our goal is to web-scrape advertisements listed on website bazos.cz, which is currently one of the most used websites for selling used cars in Czech republic. It has more than 15 000 car adds daily. On the other hand, it has really poor search options, which pretty much complicates searching for desired car based on your parameters.

We are going to code algorithm, which will scan adds for the current day, pick those, which fulfill our conditions on date and car type and save their links. Then we will go to each link and save the text of the add. Then we will try to analyze the text of the add to find our parameters. In the end, we will perform some visualizations of distribution of price and selected parameters and try to select some underprices offers.

This task might be better to perform continuouslly, for example each hour, to not overload the website. This approach might also allow for longer time period to analyze the data. But since this project should be designed as one-time run, we decided to limit the data to only current date.

In [1]:
import requests
from bs4 import BeautifulSoup
import re

In [2]:
bots = requests.get('https://auto.bazos.cz/robots.txt')
print(bots.text)

User-agent: *
Disallow: /*humkreis
Disallow: /*order=
Disallow: /*cenaod=
Disallow: /*vkm=
Disallow: /*gal=
Disallow: /suggest.php
Disallow: /suggestpsc.php
Disallow: /suggestpscinsert.php
Disallow: /overeni.php
Disallow: /detailtel.php
Disallow: /agent.php
Disallow: /favourite.php
Disallow: /kmailn.php
Disallow: /upload.php
Disallow: /zip.php
Disallow: /smazat/

User-agent: Mediapartners-Google
Disallow:

User-agent: trovitBot
Disallow: /

User-agent: AhrefsBot
Disallow: /

User-agent: pricebot
Disallow: /

User-agent: magpie-crawler
Disallow: /

User-agent: SemrushBot
Disallow: /

User-Agent: trendictionbot
Disallow: /

User-agent: SemrushBot
Disallow: /

User-Agent: trendictionbot
Disallow: /

User-agent: BLEXBot
Disallow: /

User-agent: Riddler
Disallow: /

Sitemap: https://auto.bazos.cz/sitemap.php


From the robots page we can see that our actions done in our projects are allowed, since we are not gonna use these search commands.

Here we will get the main page from car section of bazos.

In [29]:
# firstly, we define the input variable so the user can search according to their preference
user_search_input = str("octávia 4")

user_search_input = re.sub(r"[^\w\s]", '', user_search_input)
user_search_input = re.sub(r"\s+", '+', user_search_input)

number_sequence = range(0, 320, 20) #start, stop (not included), step
# we create empty list for saving the urls,
# and then append other tabs with offers (since there are only 20 offers per tab by default)
main_url_list = list()

for i in range(0,len(number_sequence)):
    url = f'https://auto.bazos.cz/{number_sequence[i]}/'
    url = url + '?hledat=' + user_search_input + '&hlokalita=&humkreis=25&cenaod=&cenado=&order='
    main_url_list.append(url)
# here we can check the list of urls for particular tabs
print(main_url_list)

['https://auto.bazos.cz/0/?hledat=octávia+4&hlokalita=&humkreis=25&cenaod=&cenado=&order=', 'https://auto.bazos.cz/20/?hledat=octávia+4&hlokalita=&humkreis=25&cenaod=&cenado=&order=', 'https://auto.bazos.cz/40/?hledat=octávia+4&hlokalita=&humkreis=25&cenaod=&cenado=&order=', 'https://auto.bazos.cz/60/?hledat=octávia+4&hlokalita=&humkreis=25&cenaod=&cenado=&order=', 'https://auto.bazos.cz/80/?hledat=octávia+4&hlokalita=&humkreis=25&cenaod=&cenado=&order=', 'https://auto.bazos.cz/100/?hledat=octávia+4&hlokalita=&humkreis=25&cenaod=&cenado=&order=', 'https://auto.bazos.cz/120/?hledat=octávia+4&hlokalita=&humkreis=25&cenaod=&cenado=&order=', 'https://auto.bazos.cz/140/?hledat=octávia+4&hlokalita=&humkreis=25&cenaod=&cenado=&order=', 'https://auto.bazos.cz/160/?hledat=octávia+4&hlokalita=&humkreis=25&cenaod=&cenado=&order=', 'https://auto.bazos.cz/180/?hledat=octávia+4&hlokalita=&humkreis=25&cenaod=&cenado=&order=', 'https://auto.bazos.cz/200/?hledat=octávia+4&hlokalita=&humkreis=25&cenaod=

In [30]:
# in the next step, we get the text of each of those tabs using the BeautifulSoup function,
# and save it as elements of the "soup_list"
soup_list = list()
for url in main_url_list:
    page = requests.get(url)
    soup_list.append(BeautifulSoup(page.text, 'html'))


In [31]:
# too long to print
print(soup_list)

[<!DOCTYPE html>
<html lang="cs">
<head>
<title>Octávia 4 bazar - Auto | Bazoš.cz</title>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="Auto - Octávia 4 bazar. Vybírejte z 319 inzerátů. Prodej snadno a rychle na Bazoši. Přes půl milionů uživatelů za den. Najděte co potřebujete." name="description"/><link href="https://auto.bazos.cz/inzeraty/oct%C3%A1via-4/" rel="canonical"/><meta content="1055875657" property="fb:admins"/>
<link href="https://www.bazos.cz/bazos60s.css" rel="stylesheet" type="text/css"/><link as="image" href="https://www.bazos.cz/obrazky/bazos.svg" rel="preload"/>
<link href="https://www.bazos.cz/bazosprint.css" media="print" rel="stylesheet" type="text/css"/>
<link href="https://www.bazos.cz/favicon.ico" rel="shortcut icon"/>
<link href="https://www.bazos.cz/obrazky/icon-192x192.png" rel="icon" sizes="192x192"/>
<link href="https://www.bazos.cz/apple-touch-icon.png" rel="apple-touch-icon"/>
<link href="https://www.bazos.cz/rss.php?

In [67]:
# musím dodelat to, abych vytahnul odsud datum inzeratu a pak ho porovnal 
# se systemovym datumem jako string (predtim sjednotit format)

# pak ve for cyklu vyse budu ukladat jenom inzeraty ktere tohle jeste splnuji....

# mozna pak jeste predelat predvoleny pocet tabs ktere prochazim podle poctu inzeratu
# (vytahnout z classy co je nahore zobrazeno)

x = soup_list[1].find('div', {'class':'inzeraty inzeratyflex'})
y = x.find('div', {'class':'inzeratynadpis'})
attribute_a = y.find('a')
attribute_span = y.find('span')
x_href = attribute_a.get('href')
x_date = str(attribute_span.text)

#TADY JSEM SKONCIL - OPRAVIT
x_date_semi_stripped = x_date.strip("]")
x_date_stripped = x_date_semi_stripped.strip("")
#x_date

print(type(x_date))

print(x_href)
print(x_date_stripped)

<class 'str'>
/inzerat/156973243/plaketka-do-volantu-laurinklement-origo.php
 - [28.8. 2022


There are 20 adds plus other page elements. Lets pick just one add to see their construction.

There are several elements, which are important to us. In this section, we are interested in add id, which we are able to extract from href in element a on the second line. Its the number after "inzerat/". We are also interested in the href itself, since it is our link to follow to the actual add. Another element important to us is h2 class = "nadpis", which contains title of the add, from which we can extract type of car. And the last interesting element is span with class "velikost10", which reffers to date uploaded.

Now we can follow the link to the actual add page to see its content.

In [8]:
# by the following code, we get the urls of each advertisement /offer/ (listed in the tabs we work with),
# and save it to "list_of_offers_url"
list_of_offers_url = list()
for element in soup_list:
    x = element.find('div', {'class':'inzeraty inzeratyflex'})
    y = x.find('div', {'class':'inzeratynadpis'})
    z = y.find('a')
    w = z.get('href')
    list_of_offers_url.append(f'https://auto.bazos.cz{w}')

print(list_of_offers_url)

['https://auto.bazos.cz/inzerat/156899528/seat-leon-st-16tdi-85kw-dsg-plna-zaruka-2-roky-zdarma.php', 'https://auto.bazos.cz/inzerat/156889523/qashqai-15-dci-rok-2011-122000-km-81-kw.php', 'https://auto.bazos.cz/inzerat/156852921/skoda-kodiaq-20tdi-110kw-dsg-lk-plna-zaruka-2-roky-zdarma.php', 'https://auto.bazos.cz/inzerat/156800462/skoda-octavia-3-16tdi-85kw-plna-zaruka-2-roky-zdarma.php', 'https://auto.bazos.cz/inzerat/156701370/hyundai-i30-14-i-benzin-6-rychlosti-puvod-cr-od-1majitele.php', 'https://auto.bazos.cz/inzerat/154947423/skoda-octavia-16-tdi-81-kw-combi-style-od-1maj-puvod-cr.php', 'https://auto.bazos.cz/inzerat/156532359/skoda-octavia-14-tsi-cng.php', 'https://auto.bazos.cz/inzerat/156590680/corolla.php', 'https://auto.bazos.cz/inzerat/156457126/vw-passat-b8-20tdi-110kw-dsg-plna-zaruka-2-roky-zdarma.php', 'https://auto.bazos.cz/inzerat/156890473/skoda-superb-elegance-25tdi-manual-nova-stkbixenontop.php', 'https://auto.bazos.cz/inzerat/156352222/vw-passat-20-tdi-103-kw-com

In [ ]:
#similar as above for class = 'velikos10', 'nadpis', ...
# also the code below will be automatized using for loops, OOP, and etc.

In [ ]:
# by the following code, we get the source text of each advertisement, again using the BeautifulSoup function,
# then we save that into a list "list_of_source_text_of_adv"
list_of_source_text_of_adv = list()
for url_element in list_of_offers_url:
    add_page = requests.get(url_element)
    soup_add = BeautifulSoup(add_page.text, 'html')
    list_of_offers_url.append(soup_add)

In [14]:
# we will print just first element to check the content, since doing that for entire list would take too much time,
# and probably even the output  would be too large to print
print(list_of_source_text_of_adv)

[]


In [9]:
#continue here??? - maybe start here the text mining part? - if the code above works,
# I (Daniel) will make it as OOP using class
add_page = requests.get('https://auto.bazos.cz/inzerat/151109412/volkswagen-golf-gtd-135kw-manual-vyhrev-navi-led.php')
soup_add = BeautifulSoup(add_page.text, 'html')
print(soup_add.find('div', {'class':'popisdetail'}))

None


Here we can see the description, which we will use to extract parameters of the car. This will be one of the most difficult parts of the project to extract the relevant text in unformated text. There is no official format to the text, so we will have to find a way how to extract this information from various formats.

After retrieving these informations, we will use analysis of the data got from this process.